# Flowmetrics – Impact-Augmented Knowledge Graph Construction

This notebook constructs the Impact-Augmented Knowledge Graph, the central data structure underpinning the Flowmetrics framework. It documents the end-to-end pipeline for generating a structured dataset of societal research impact trajectories by integrating heterogeneous data sources into an RDF graph suitable for AI-driven impact modelling.

### Objective

To automate the extraction, semantic alignment, and structuring of research topic pairs and their associated impact signals — enabling scalable modelling of how research impact unfolds across time, platforms, and audiences.

### Structure

#### 1. Data Collection Pipeline  
Harvests impact evidence through API integration with three key platforms:  
- **arXiv** – source of metadata for topic extraction and co-occurrence modelling  
- **Altmetric** – provider of online attention signals (e.g., news, social media, blogs)  
- **CrossRef** – supplier of citation-based and policy-linked influence metrics

The pipeline operates on a curated corpus of 12,350 computer science preprints (2000–2024), spanning major subfields such as machine learning, natural language processing, computer vision, and artificial intelligence. Papers were selected for topical diversity, metadata completeness, and coverage across at least one impact platform.

# Table of Contents
- [1. Data collection pipeline: automating data extraction](#section-1)
  - [1.1 API Integration](#subsection-11)
     - [1.1.1 Altmetric Data](#subsection-111)
     - [1.1.2 CrossRef Data](#subsection-112)
     - [1.1.3 arXiv Data](#subsection-113)
  - [1.2 Impact Trajectory Matching](#subsection-12)
     - [1.2.1 Nodes of topics](#subsection-121)
     - [1.2.2 Edges of impact](#subsection-122)
     - [1.2.3 Impact-augmented knowledge graph](#subsection-123)

In [54]:
import json
import nltk
import requests
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
from pyvis.network import Network
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from cso_classifier import CSOClassifier
from pathlib import Path

In [55]:
# project directory
project_dir = Path(".").resolve().parent

## 1.1 API Integration  
### 1.1.1 Altmetric Data

Altmetric data captures the online attention and engagement surrounding scholarly publications, providing insight into the broader impact of research beyond traditional citation metrics. It reflects how a paper is being discussed, shared, and interacted with across a range of platforms including news outlets, blogs, Twitter, Facebook, Reddit, Wikipedia, and policy documents.

This data helps construct a multidimensional view of research visibility and societal relevance — spanning public discourse, academic engagement, and policy uptake. As such, Altmetric indicators are increasingly valuable for researchers, institutions, and funders aiming to understand and quantify how research resonates across different audiences and sectors.

In [70]:
def get_altmetric_data(doi):
    if not doi:
        return [None] * 12  # Return None for all counts if DOI is invalid

    url = f"https://api.altmetric.com/v1/doi/{doi}"
    
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Use dictionary's get method to return None for missing keys instead of defaulting to 0
            return [
                data.get("score"),
                data.get("cited_by_posts_count"),
                data.get("cited_by_tweeters_count"),
                data.get("cited_by_news_outlets_count"),
                data.get("cited_by_blogs_count"),
                data.get("cited_by_rdts_count"),
                data.get("cited_by_fbwalls_count"),
                data.get("cited_by_patents_count"),
                data.get("cited_by_wikipedia_count"),
                data.get("cited_by_policy_count"),
                data.get("cited_by_mendeley_count"),
                data.get("cited_by_videos_count")
            ]
        
        # Handle different status codes
        elif response.status_code == 401:
            print("Unauthorized: Check your API key or permissions.")
        elif response.status_code == 429:
            print("Rate limit exceeded. Please try again later.")
        else:
            pass

    except requests.exceptions.RequestException as e:
        print(f"Network or request error: {e}")
    
    # Return None values if the request fails or DOI is invalid
    return [None] * 12

### 1.1.2 CrossRef Data

CrossRef data provides extensive metadata and citation information for scholarly works, including journal articles, books, conference proceedings, datasets, and other research outputs. It plays a critical role in enhancing the discoverability and traceability of academic content through the use of persistent identifiers such as Digital Object Identifiers (DOIs).

Within the Flowmetrics framework, CrossRef serves as a key source of citation-based impact signal. These signals help trace the academic and institutional reach of research over time, offering a complementary view to socially-driven metrics and enabling a more comprehensive understanding of scholarly influence.

In [71]:
def get_citation_count_from_crossref(doi):
    if not doi:
        return None  # Return None immediately if DOI is invalid

    url = f"https://api.crossref.org/works/{doi}"

    try:
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            return data.get('message', {}).get('is-referenced-by-count', 0)

        # Handle common error status codes
        if response.status_code == 404:
            pass
        elif response.status_code == 429:
            print("Rate limit exceeded. Please try again later.")
        else:
            pass

    except requests.exceptions.RequestException as e:
        print(f"Network or request error: {e}")

    return None

### 1.1.3 arXiv Data

arXiv is a widely used preprint repository that offers open access to scholarly articles across a broad range of disciplines, including computer science, physics, mathematics, statistics, electrical engineering, quantitative biology, and economics.

In the Flowmetrics framework, arXiv serves as the primary source for research metadata, enabling the extraction of topics and co-occurrence patterns. This metadata provides the structural backbone for identifying topic pairs and aligning them with downstream impact signals collected from Altmetric and CrossRef.

In [72]:
DATASET_FILE = project_dir / "data" / "arxiv-metadata-oai-snapshot.json"

In [73]:
category_map = {
    'cs.AI': 'Artificial Intelligence',
    'cs.CL': 'Computation and Language',
    'cs.CV': 'Computer Vision and Pattern Recognition',
    'cs.DS': 'Data Structures and Algorithms',
    'cs.ET': 'Emerging Technologies',
    'cs.HC': 'Human-Computer Interaction',
    'cs.IR': 'Information Retrieval',
    'cs.NE': 'Neural and Evolutionary Computing',
    'cs.LG': 'Machine Learning'
}

In [74]:
# Initialize wordnet lemmatizer
wnl = WordNetLemmatizer()

def remove_stop_words(sentence):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(sentence)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return ' '.join(filtered_sentence).replace(',','')

def lemmatizer_word(word):
    return wnl.lemmatize(word)

def lemmatizer(sentence):
    tokens = nltk.word_tokenize(sentence)
    lemmatized_tokens = [lemmatizer_word(token) for token in tokens]
    return " ".join(lemmatized_tokens)

def get_metadata():
    with open(DATASET_FILE, 'r') as f:
        for line in f:
            yield line

In [75]:
data = {
    'dois': [], 'titles': [], 'abstracts': [], 'years': [], 'categories': [],
    'citations_crossref': [], 'citations_wos': [], 'altmetric_score': [], 
    'all_mentions_counts': [], 'twitter_counts': [], 'news_counts': [],
    'blogs_counts': [], 'reddit_counts': [], 'facebook_counts': [],
    'patents_counts': [], 'wiki_counts': [], 'policy_counts': [],
    'mendeley_counts': [], 'video_counts': [], 'views': [], 'downloads': []
}

In [ ]:
metadata = get_metadata()

# Process metadata and populate the data dictionary
for paper in metadata:
    paper_dict = json.loads(paper)
    ref = paper_dict.get('journal-ref')
    doi = paper_dict.get('doi')
    try:
        year = int(ref[-4:])
        if 2000 < year <= 2024:
            category = paper_dict.get('categories').split(" ")[0]
            data['categories'].append(category_map.get(category, 'Unknown'))  # Handle missing category mapping
            data['years'].append(year)
            data['titles'].append(lemmatizer(remove_stop_words(paper_dict.get('title', ''))))
            data['abstracts'].append(lemmatizer(remove_stop_words(paper_dict.get('abstract', ''))))
            data['dois'].append(doi)
            data['citations_crossref'].append(get_citation_count_from_crossref(doi))
            
            altmetric_data = get_altmetric_data(doi)
            data['altmetric_score'].append(altmetric_data[0])
            data['all_mentions_counts'].append(altmetric_data[1])
            data['twitter_counts'].append(altmetric_data[2])
            data['news_counts'].append(altmetric_data[3])
            data['blogs_counts'].append(altmetric_data[4])
            data['reddit_counts'].append(altmetric_data[5])
            data['facebook_counts'].append(altmetric_data[6])
            data['patents_counts'].append(altmetric_data[7])
            data['wiki_counts'].append(altmetric_data[8])
            data['policy_counts'].append(altmetric_data[9])
            data['mendeley_counts'].append(altmetric_data[10])
            data['video_counts'].append(altmetric_data[11])
    except (KeyError, ValueError, TypeError) as e:
        pass

# Return the lengths of the lists to check if they match
len(data['dois']), len(data['titles']), len(data['abstracts']), len(data['years']), len(data['categories']), len(data['citations_crossref']), len(data['altmetric_score']), len(data['views']), len(data['downloads'])

In [ ]:
# Create a DataFrame
df = pd.DataFrame({
    'DOI': dois,
    'Title': titles,
    'Abstract': abstracts,
    'Year': years,
    'Category': categories,
    'Citation_crossref': citations_crossref,
    'Altmetric_score': altmetric_score,
    'All_mentions': all_mentions_counts,
    'Twitter': twitter_counts,
    'News': news_counts,
    'Blogs': blogs_counts,
    'Reddit': reddit_counts,
    'Facebook': facebook_counts,
    'Patents': patents_counts,
    'Policy': policy_counts,
    'Mendeley': mendeley_counts,
    'Wikipedia': wiki_counts,
    'Videos': video_counts,
})

In [ ]:
# save pre-processed files
#DATASET_PATH = project_dir / "data" / "processed" / "flow_dataset.csv"
#df.to_csv(DATASET_PATH, index=False)

## 1.2 Impact Trajectory Matching

In [ ]:
df

In [ ]:
df["Category"].value_counts()

### 1.2.1 Nodes of Topics

The nodes of the knowledge graph represent high-quality research topics extracted from arXiv metadata. To identify these topics, we used the **CSO Classifier** — an unsupervised tool that assigns concepts from the **Computer Science Ontology (CSO)** based on paper titles, abstracts, and keywords.

The CSO Classifier integrates two components:  
- A **syntactic module**, which detects explicitly mentioned concepts  
- A **semantic module**, which leverages part-of-speech tagging and word embeddings to infer related concepts

Outputs from both modules are merged to generate a candidate topic list for each paper. To consolidate fine-grained variations, we applied a **frequency-based clustering strategy**, assigning each paper to its most frequently associated concept across the corpus. This allowed for the grouping of semantically similar papers under unified topic identifiers without manual filtering.

The resulting pipeline generated **156 coherent, domain-relevant topic nodes**, covering diverse areas of computer science.

In [ ]:
!pip install CSOClassifier

In [ ]:
cc = CSOClassifier(modules = "both", enhancement = "all", explanation = False)
results = list()

In [ ]:
def create_paper_dict(row):
    paper = {
        "title": row["Title"],
        "abstract": row["Abstract"]
    }
    return cc.run(paper)

In [ ]:
# Apply function to create the 'cso_topics' column
df["CSO_classifier"] = df.apply(create_paper_dict, axis=1)

In [ ]:
for _, row in df.head(10).iterrows():
    print(f"Category: {row['Category']}")
    print(f"Title: {row['Title']}")
    print(f"Abstract: {row['Abstract']}")
    print(f"CSO Topics: \n{json.dumps(row['CSO_classifier'], indent=4)}\n")
    print("="*80)

In [ ]:
# Extract the "union" list from each dictionary
df["Research_concepts"] = df["CSO_classifier"].apply(lambda x: x.get("union", []))

In [ ]:
df["Super_topics"] = df["CSO_classifier"].apply(lambda x: x.get("enhanced", []))

In [ ]:
#df["Research_concepts"] = df["Research_concepts"].apply(ast.literal_eval)
all_concepts = df["Research_concepts"].apply(lambda x: x).explode().tolist()

In [ ]:
# Step 1: Count the frequency of each concept across all papers
concept_counts = pd.Series(all_concepts).value_counts()

In [ ]:
# Step 2: Assign the most frequent concept as the cluster for each paper
df["Name"] = df["Research_concepts"].apply(
    lambda concepts: max(concepts, key=lambda concept: concept_counts.get(concept, 0))
)
# Step 3: Generate unique Topic IDs starting from 0
df["Topic"] = df["Name"].map({name: idx for idx, name in enumerate(df_dataset["Name"].unique())})
df

In [ ]:
# Ensure feedback columns exist in df, even if filled with 0.0
for col in ["Peers", "Expert"]:
    if col not in df.columns:
        df[col] = 0.0

### 1.2.2 Edges of Impact

Edges in the Impact-Augmented Knowledge Graph represent impact-bearing relationships between research topics. An edge is established when two topics co-occur in at least one paper and share a common CSO concept. These relationships are encoded as RDF triples of the form *(TopicPair, hasStageImpact, BlankNode)*.

Each `hasStageImpact` predicate is instantiated with a stage-specific property (e.g., `flow:hasReachImpact`, `flow:hasInfluenceImpact`). The associated `BlankNode` stores metadata about the impact source, including:  
- `flow:platform`: the platform generating the signal (e.g., Twitter, Facebook)  
- `flow:score`: the aggregated and normalised impact score for that platform and stage

#### Aggregation Strategy

To produce these edge-level scores, we applied a two-step aggregation process:  
1. **Normalisation:** Raw scores were independently normalised per platform and dimension to account for scale differences (e.g., Altmetric vs. CrossRef).  
2. **Summation:** For each topic pair and stage, we summed the normalised scores across all platforms associated with that dimension.

#### Example

Consider the topic pair `Semantics` and `Language Model`, which co-occur and share the CSO concept `language model`. Their Reach impact is encoded as the following RDF triples:  
*(flow:pair_15_131, flow:hasReachImpact, :reach1)*,  
*(:reach1, flow:platform, "Twitter")*,  
*(:reach1, flow:score, 0.1321)*,  
*(:reach2, flow:platform, "Facebook")*,  
*(_:reach2, flow:score, 0.0972)*

In [ ]:
def normalize(series, range_min, range_max):
    min_val = series.min()
    max_val = series.max()
    return ((series - min_val) / (max_val - min_val) * (range_max - range_min) + range_min).astype(int)

In [ ]:
def get_top_10_concepts(concepts):
    concepts_count = pd.Series(concepts).value_counts()
    top_10_concepts = concepts_count.nlargest(10).index.tolist()
    top_10_counts = concepts_count.nlargest(10).values.tolist()
    top_10_concepts_with_counts = list(zip(top_10_concepts, top_10_counts))
    return top_10_concepts_with_counts

In [ ]:
# Flowmetrics labeling
reach_columns = ["Twitter", "Facebook", "Wikipedia"]
engagement_columns = ["Blogs", "Reddit", "Videos", "Mendeley"]
feedback_columns = ["Peers", "Expert"]
influence_columns = ["News", "Citation_crossref"]
outcomes_columns = ["Policy", "Patents"]

In [ ]:
agg_scores = df_dataset.groupby(["Topic", "Name"]).agg({
    "Twitter": "sum",
    "Facebook": "sum",
    "Reddit": "sum",
    "Wikipedia": "sum",
    "Blogs": "sum",
    "Videos": "sum",
    "Mendeley": "sum",
    "Peers": "sum",
    "Expert": "sum",
    "News": "sum",
    "Citation_crossref": "sum",
    "Policy": "sum",
    "Patents": "sum",
    "Research_concepts": lambda x: [concept for sublist in x for concept in sublist]
}).reset_index()

agg_scores['Count'] = df_dataset.groupby(["Topic", "Name"]).size().values
agg_scores['Count_norm'] = normalize(df_dataset.groupby(["Topic", "Name"]).size().values, 40, 100)
agg_scores["Reach"] = agg_scores[reach_columns].apply(lambda row: list(zip(reach_columns, row)), axis=1)
agg_scores["Engagement"] = agg_scores[engagement_columns].apply(lambda row: list(zip(engagement_columns, row)), axis=1)
agg_scores["Feedback"] = agg_scores[feedback_columns].apply(lambda row: list(zip(feedback_columns, row)), axis=1)
agg_scores["Influence"] = agg_scores[influence_columns].apply(lambda row: list(zip(influence_columns, row)), axis=1)
agg_scores["Outcome"] = agg_scores[outcomes_columns].apply(lambda row: list(zip(outcomes_columns, row)), axis=1)
agg_scores["Representation"] = agg_scores["Research_concepts"].apply(get_top_10_concepts)

In [ ]:
# Remove topics where the 'Representation' list has fewer than 10 concepts
agg_scores = agg_scores[agg_scores["Representation"].apply(lambda x: len(x) == 10)].reset_index(drop=True)
agg_scores

In [ ]:
topics = {}

for idx, row in agg_scores.iterrows():
    topics[idx] = {
        'topic': row['Name'],
        'cluster_size': row['Count_norm'],
        'reach': row['Reach'],
        'engagement': row['Engagement'],
        'feedback': row['Feedback'],
        'influence': row['Influence'],
        'outcome': row['Outcome'],
        'words': row['Representation']  # List of tuples (concept, count)
    }
#import pprint
#pprint.pprint(dict(list(topics.items())[:10]))

### 1.2.3 Impact-augmented knowledge graph

In [ ]:
def normalize_edge_weights(G, weight_attr="width", new_range=(0, 1)):
    a, b = new_range
    weights = [G[u][v][weight_attr] for u, v in G.edges]
    min_weight = min(weights)
    max_weight = max(weights)
    for u, v in G.edges:
        original_weight = G[u][v][weight_attr]
        normalized_weight = a + ((original_weight - min_weight) * (b - a)) / (max_weight - min_weight)
        G[u][v][weight_attr] = round(normalized_weight, 2)

In [ ]:
G = nx.Graph()

for topic, value in topics.items():
    topic_node = f"Topic {topic}"
    G.add_node(topic_node, size=(value['cluster_size']), label=topic_node, type='topic')
    
    for word, _ in value.get("words", [])[:10]: #[:100]
        G.add_node(word, size=10, label=word, type='leaf')
        G.add_edge(topic_node, word, weight=1)

overlap_leafs = []
for topic1, value1 in topics.items():
    for topic2, value2 in topics.items():
        if topic1 != topic2:
            words1 = set([word[0] for word in value1.get("words", [])])
            words2 = set([word[0] for word in value2.get("words", [])])
            common_words = words1.intersection(words2)
            if common_words:
                overlap_leafs.append(next(iter(common_words)))
                G.add_edge(f"Topic {topic1}", f"Topic {topic2}", color="#4caf50", weight=len(common_words) + 
                           float(sum(value for _, value in value1.get("reach", []))) + 
                           float(sum(value for _, value in value2.get("reach", []))) + 
                           float(sum(value for _, value in value1.get("engagement", []))) + 
                           float(sum(value for _, value in value2.get("engagement", []))) +
                           float(sum(value for _, value in value1.get("feedback", []))) + 
                           float(sum(value for _, value in value2.get("feedback", []))) +
                           float(sum(value for _, value in value1.get("influence", []))) + 
                           float(sum(value for _, value in value2.get("influence", []))) +
                           float(sum(value for _, value in value1.get("outcome", []))) + 
                           float(sum(value for _, value in value2.get("outcome", []))))

# Normalize weights
normalize_edge_weights(G, weight_attr="weight", new_range=(1, 20))

In [ ]:
net = Network(height="800px", width="100%", notebook=True, bgcolor="white", font_color="black") ##f0f0f0
net.from_nx(G)

for node in net.nodes:
    if node['type'] == "topic":
        node['borderWidth'] = 4
        node['shadow'] = True
        node['color'] = "#9e9e9e"
    else:
        if node['id'] in overlap_leafs:
            node['borderWidth'] = 3
            node['shadow'] = True
            node['color'] = "#ff9800"
        else:
            node['borderWidth'] = 3
            node['shadow'] = True
            node['color'] = "#616161"

for edge in net.edges:
    weight = edge.get('width', 1)
    edge['label'] = str(weight)
    edge['font'] = {'size': 8}
    
net.force_atlas_2based()
net.show("G_topic_knowledge_graph.html")